In [84]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession


spark = SparkSession.builder.getOrCreate()

# fractions = [[x,y] for x in range(10) for y in range(1, 10)]
# frac_df = spark.createDataFrame(fractions, ["numerator", "denumerator"])
# frac_df = frac_df.select(F.array(F.col("numerator"), F.col("denumerator")).alias("fraction"))
# frac_df.show()

In [85]:
# from fractions import Fraction
# from typing import Tuple, Optional

# Frac = Tuple[int,int]

# def py_reduce_fraction(frac: Frac) -> Optional[Frac]:
#     num, denum = frac
#     if denum:
#         answer = Fraction(num, denum)
#         return answer.numerator, answer.denominator       
#     return False

In [86]:
# logs = spark.read.csv("../Data/broadcast_logs/BroadcastLogs_2018_Q3_M8_sample.csv",
#                       sep = "|",
#                       header=True,
#                       inferSchema=True,
#                       timestampFormat="yyyy-MM-dd")

In [87]:
df_titanic = spark.read.csv("../Data/titanic.csv", inferSchema=True, header=True)

In [88]:
df_titanic.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [89]:
def extract_tiltie(name):
    return name.split(", ")[-1].split()[0]

def is_common_title(title):
    return 1 if title in ["Miss.", "Mr.", "Mrs.", "Master"] else 0
    
def remap_gender(gender):
    return 1 if gender == "male" else 0

@F.udf()
def has_cabin(cabin):
    return 1 if cabin else 0

@F.udf()
def replace_missing_age(age):
    return age if age else 0

In [90]:
extract_tiltie = F.udf(extract_tiltie)
is_common_title = F.udf(is_common_title)
remap_gender = F.udf(remap_gender)
# has_cabin = F.udf(has_cabin)
# replace_missing_age = F.udf(replace_missing_age)

In [91]:
df_titanic = df_titanic.withColumn("extract_tiltie", extract_tiltie(F.col("Name")))
df_titanic = df_titanic.withColumn("is_common_title", is_common_title(F.col("extract_tiltie")))
df_titanic = df_titanic.withColumn("has_cabin", has_cabin(F.col("Cabin")))
df_titanic = df_titanic.withColumn("replace_missing_age", replace_missing_age(F.col("Age")))

In [92]:
df_titanic.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+--------------+---------------+---------+-------------------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|extract_tiltie|is_common_title|has_cabin|replace_missing_age|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+--------------+---------------+---------+-------------------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|           Mr.|              1|        0|               22.0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|          Mrs.|              1|        1|               38.0|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|      